In [1]:
! git clone https://github.com/nbtpj/LMRecTest.git
import os
os.chdir('LMRecTest/')
# ! pip install -r requirements.txt
! ls
# !mkdir ./cache/
# !cp -R /kaggle/input/movilenxamazonecomment/* ./cache/
# !ls cache

Cloning into 'LMRecTest'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 122 (delta 75), reused 85 (delta 38), pack-reused 0
Receiving objects: 100% (122/122), 32.25 KiB | 2.93 MiB/s, done.
Resolving deltas: 100% (75/75), done.
BARTRec.py  env_config.py  metrics.py	     runlmrec.ipynb
GPTRec.py   make_data.py   requirements.txt  test_MLRec.py


In [21]:
# data_dir = "/kaggle/input/movilenxamazonecomment"
from datasets import load_dataset, DatasetDict

subsets = ['movielens-1m-movies', 'movielens-1m-movies', ]
    
unified_dataset = DatasetDict({
    k: load_dataset(f'nbtpj/{k}') for k in subsets
})
sampled_prompt_movilens_dataset = load_dataset(f'nbtpj/6k-sample')['train']
sampled_prompt_movilens_dataset = sampled_prompt_movilens_dataset.select(range(1000))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
# !python make_data.py 4000

In [23]:
from env_config import TERM_TO_ESTIMATE, RATINGS, movielen_feat_map

test_scenario = {
    'biography only': ['bio prompt'],
    'bio + top 10 nearest ratings': ['bio prompt', 'history prompt'],
    'bio + 10 nearest ratings + history overview': ['bio prompt', 'history prompt', 'history overview prompt'],
    'bio + 10 nearest ratings + history overview + age-group': ['bio prompt', 'history prompt', 'history overview prompt', 'age-group prompt'],
    'bio + 10 nearest ratings + history overview + age-group + next-category': ['bio prompt', 'history prompt', 'history overview prompt', 'age-group prompt', 'cross-cate prompt'],
}

def film_promt(film_meta: dict) -> dict:
    title = None
    try:
        title = film_meta['movie_title'].decode('utf-8')
    except:
        pass
    categories = []
    try:
        categories = [movielen_feat_map['movie_genres'][k] for k in film_meta['movie_genres']]
    except:
        pass

    return {
        'target prompt': f'The movie named {title} is categorized as {", ".join(categories)}.{TERM_TO_ESTIMATE}'
    }

def map2ctx(rating, test_scenario, targets_ids):
    r = {}
    for name, cols_to_gather in test_scenario.items():
        try:
            r[name] = [rating[col] for col in cols_to_gather]
            r[name] = ' '.join(r[name])
        except:
            pass
    r.update({
        'user_rating': rating['user_rating'],
        'expected': targets_ids.index(rating['movie_id'])
    })
    return r

target_details = sampled_prompt_movilens_dataset.map(film_promt)
target_details

  0%|          | 0/1000 [00:00<?, ?ex/s]

Dataset({
    features: ['bucketized_user_age', 'movie_genres', 'movie_id', 'movie_title', 'timestamp', 'user_gender', 'user_id', 'user_occupation_label', 'user_occupation_text', 'user_rating', 'user_zip_code', 'bio prompt', 'history prompt', 'history overview prompt', 'job-group prompt', 'age-group prompt', 'gender-group prompt', 'region-group prompt', 'cross-movie prompt', 'cross-cate prompt', 'target prompt'],
    num_rows: 1000
})

In [24]:
RATINGS

['This user rate this movie 1 out of 5.',
 'This user rate this movie 2 out of 5.',
 'This user rate this movie 3 out of 5.',
 'This user rate this movie 4 out of 5.',
 'This user rate this movie 5 out of 5.']

In [25]:
from env_config import device
device

'cuda:0'

# Test Recommender Scenario
___

In [26]:
import numpy as np

targets = target_details['target prompt'] # all possible selections
targets_ids = target_details['movie_id']
context_selection = sampled_prompt_movilens_dataset.map(map2ctx, 
                                                        fn_kwargs = {'test_scenario':test_scenario, 'targets_ids': targets_ids},
                                                        remove_columns=sampled_prompt_movilens_dataset.column_names)
expected = np.array(context_selection['expected'])
context_selection

  0%|          | 0/1000 [00:00<?, ?ex/s]

Dataset({
    features: ['user_rating', 'biography only', 'bio + top 10 nearest ratings', 'bio + 10 nearest ratings + history overview', 'bio + 10 nearest ratings + history overview + age-group', 'bio + 10 nearest ratings + history overview + age-group + next-category', 'expected'],
    num_rows: 1000
})

In [28]:
%env CUDA_LAUNCH_BLOCKING=1
from transformers import BartTokenizer, BartForConditionalGeneration, GPT2Tokenizer, GPT2LMHeadModel
from metrics import *
from tqdm import tqdm
from BARTRec import rank_with_bart
from GPTRec import rank_with_gpt

available_selections = target_details['target prompt']
rank_method_names = ['BART', 'GPT']
rank_methods = [rank_with_bart, rank_with_gpt]
models = [BartForConditionalGeneration.from_pretrained('facebook/bart-base').to(device), GPT2LMHeadModel.from_pretrained('gpt2').to(device)]
tokenizers = [BartTokenizer.from_pretrained('facebook/bart-base'), GPT2Tokenizer.from_pretrained('gpt2')]
tokenizers[-1].add_special_tokens({'pad_token': '[PAD]'})
size = 20
results = []
total = len(test_scenario) * len(rank_methods) * len(context_selection)
with tqdm(test_scenario, total=total, desc='Processing') as process:
    for scenario in process:
        if scenario in context_selection.column_names:
            contexts = context_selection[scenario]
            for rank_method, model, tokenizer, name in zip(rank_methods, models, tokenizers, rank_method_names):
                predictions = rank_method(model=model, tokenizer=tokenizer,
                                    contexts=contexts, available_selections=available_selections,
                                    term_to_estimate = TERM_TO_ESTIMATE,
                                    verbose=process,
                                    batch_size=16)
                eval_rs = MMRandR(predictions, expected, size=20)
                results.append({'model': name, 'scenario': scenario, **eval_rs})
len(results)

env: CUDA_LAUNCH_BLOCKING=1


Processing:   0%|          | 0/10000 [01:20<?, ?it/s]           


KeyboardInterrupt: 

In [ ]:
import pandas as pd
df = pd.DataFrame(results)
df

# Test Rating-prediction Scenario
___

In [29]:
import numpy as np

targets = target_details['target prompt'] # all possible selections
targets_ids = target_details['movie_id']
context_selection = sampled_prompt_movilens_dataset.map(map2ctx, 
                                                        fn_kwargs = {'test_scenario':test_scenario, 'targets_ids': targets_ids},
                                                        remove_columns=sampled_prompt_movilens_dataset.column_names)
expected = np.array(context_selection['user_rating'])
context_selection

  0%|          | 0/1000 [00:00<?, ?ex/s]

Dataset({
    features: ['user_rating', 'biography only', 'bio + top 10 nearest ratings', 'bio + 10 nearest ratings + history overview', 'bio + 10 nearest ratings + history overview + age-group', 'bio + 10 nearest ratings + history overview + age-group + next-category', 'expected'],
    num_rows: 1000
})

In [30]:
from transformers import BartTokenizer, BartForConditionalGeneration, GPT2Tokenizer, GPT2LMHeadModel
from metrics import *
from tqdm import tqdm
from BARTRec import rank_with_bart
from GPTRec import rank_with_gpt
from env_config import device


available_selections = RATINGS
rank_method_names = ['BART', 'GPT']
rank_methods = [rank_with_bart, rank_with_gpt]
models = [BartForConditionalGeneration.from_pretrained('facebook/bart-base').to(device), GPT2LMHeadModel.from_pretrained('gpt2').to(device)]
tokenizers = [BartTokenizer.from_pretrained('facebook/bart-base'), GPT2Tokenizer.from_pretrained('gpt2')]
results = []
total = len(test_scenario) * len(rank_methods) * len(context_selection)

with tqdm(test_scenario, total=total, desc='Processing') as process:
    for scenario in process:
        if scenario in context_selection.column_names:
            contexts = context_selection[scenario]
            for rank_method, model, tokenizer, name in zip(rank_methods, models, tokenizers, rank_method_names):
                predictions = rank_method(model=model, tokenizer=tokenizer,
                                    contexts=contexts, available_selections=available_selections,
                                    verbose=process,
                                    batch_size=8,
                                    term_to_estimate = None)
                predictions = predictions[..., 0] + 1 # convert from index to rating
                eval_rs = rmse(predictions, expected)
                results.append({'model': name, 'scenario': scenario, **eval_rs})
len(results)

Processing:   0%|          | 5/10000 [17:53<595:58:26, 214.66s/it]


10

In [31]:
import pandas as pd
df = pd.DataFrame(results)
df

,model,scenario,rmse
0,BART,biography only,1.222293
1,GPT,biography only,2.287575
2,BART,bio + top 10 nearest ratings,2.493191
3,GPT,bio + top 10 nearest ratings,1.850405
4,BART,bio + 10 nearest ratings + history overview,2.390397
5,GPT,bio + 10 nearest ratings + history overview,2.252554
6,BART,bio + 10 nearest ratings + history overview + ...,2.477701
7,GPT,bio + 10 nearest ratings + history overview + ...,2.527251
8,BART,bio + 10 nearest ratings + history overview + ...,2.403955
9,GPT,bio + 10 nearest ratings + history overview + ...,2.207261
